In [2]:
from backend_logic import run_layout_process

In [13]:
# backend_logic.py

import os
import re
import random
from pathlib import Path
from PIL import Image, ImageDraw, ImageFont
import rectpack
import openpyxl

# ... (Tutte le funzioni da get_page_dimensions a natural_sort_key sono identiche alla versione precedente) ...
# Le includo per completezza.

def get_page_dimensions(page_size_str_px):
    try:
        width, height = map(int, page_size_str_px.split('x'))
        return (width, height)
    except ValueError:
        raise ValueError(f"Formato dimensione pagina non valido: '{page_size_str_px}'. Usare 'LARGHEZZAxALTEZZA'.")

def load_metadata(filepath, status_callback=print):
    # ... (identica)
    if not filepath: return None
    status_callback(f"Caricamento metadati da: {filepath}...")
    try:
        workbook = openpyxl.load_workbook(filepath)
        sheet = workbook.active
        metadata = {}
        header = [cell.value for cell in sheet[1]]
        for row in sheet.iter_rows(min_row=2, values_only=True):
            if row and row[0]: metadata[row[0]] = {header[i]: row[i] for i in range(1, len(row))}
        status_callback(f"Caricati metadati per {len(metadata)} elementi.")
        return metadata
    except FileNotFoundError: status_callback(f"Attenzione: File metadati '{filepath}' non trovato."); return None
    except Exception as e: status_callback(f"Errore nel caricamento del file Excel: {e}"); return None

def load_images_with_info(folder_path, status_callback=print):
    # ... (identica)
    image_data, supported_formats = [], ('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff')
    status_callback(f"Caricamento immagini da: {folder_path}...")
    if not os.path.isdir(folder_path): raise FileNotFoundError(f"'{folder_path}' non esiste.")
    for filename in sorted(os.listdir(folder_path)):
        if filename.lower().endswith(supported_formats):
            try:
                filepath = os.path.join(folder_path, filename)
                img = Image.open(filepath)
                image_data.append({'img': img.copy(), 'name': filename})
                img.close()
            except IOError: status_callback(f"Attenzione: Impossibile caricare {filename}.")
    status_callback(f"Caricati {len(image_data)} immagini."); return image_data

def add_captions_to_images(image_data, metadata, font_size, caption_padding, status_callback=print):
    # ... (identica)
    status_callback("Aggiunta didascalie alle immagini...")
    try: font = ImageFont.truetype("arial.ttf", font_size)
    except IOError: font = ImageFont.load_default()
    for data in image_data:
        img = data['img']
        caption_lines = [data['name']]
        img_metadata = metadata.get(data['name']) if metadata else None
        if img_metadata:
            for key, value in img_metadata.items():
                if value is not None: caption_lines.append(f"{key}: {value}")
        full_caption_text = "\n".join(caption_lines)
        temp_draw = ImageDraw.Draw(Image.new('RGB', (1, 1)))
        text_bbox = temp_draw.multiline_textbbox((0, 0), full_caption_text, font=font)
        text_width, text_height = text_bbox[2] - text_bbox[0], text_bbox[3] - text_bbox[1]
        new_height = img.height + text_height + caption_padding * 2
        new_width = max(img.width, text_width + caption_padding * 2)
        captioned_img = Image.new('RGB', (new_width, new_height), 'white')
        img_paste_x = (new_width - img.width) // 2
        captioned_img.paste(img, (img_paste_x, 0))
        draw = ImageDraw.Draw(captioned_img)
        text_x = (new_width - text_width) // 2
        text_y = img.height + caption_padding
        draw.multiline_text((text_x, text_y), full_caption_text, font=font, fill="black", align="center")
        data['img'] = captioned_img
    return image_data

def natural_sort_key(s):
    # ... (identica)
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', s)]

def scale_images(image_data, scale_factor, status_callback=print):
    # ... (identica)
    if scale_factor == 1.0: return image_data
    status_callback(f"Applicazione scala: {scale_factor}x")
    for data in image_data:
        new_width, new_height = int(data['img'].width * scale_factor), int(data['img'].height * scale_factor)
        data['img'] = data['img'].resize((new_width, new_height), Image.Resampling.LANCZOS)
    return image_data

def create_scale_bar(target_cm, pixels_per_cm, scale_factor, status_callback=print):
    """
    MODIFICATA: La logica ora è basata sul rapporto pixel/cm fornito dall'utente.
    Il parametro DPI non è più necessario per questo calcolo.
    """
    status_callback(f"Creazione scala grafica per rappresentare {target_cm} cm...")
    try:
        font = ImageFont.truetype("arial.ttf", 14)
    except IOError:
        font = ImageFont.load_default()
    
    # Calcola la larghezza in pixel della barra basandosi sulla scala reale fornita
    bar_width_px = int(target_cm * pixels_per_cm * scale_factor)
    
    bar_height_px = 10
    total_height = bar_height_px + 20
    
    bar_img = Image.new('RGBA', (bar_width_px + 40, total_height), (0, 0, 0, 0))
    draw = ImageDraw.Draw(bar_img)

    # Disegna la barra e le etichette
    num_segments = int(target_cm)
    if num_segments == 0: num_segments = 1
    segment_width = bar_width_px / num_segments
    for i in range(num_segments):
        color = "black" if i % 2 == 0 else "white"
        x0, x1 = i * segment_width, (i + 1) * segment_width
        draw.rectangle([x0, 0, x1, bar_height_px], fill=color, outline="black")
    
    draw.text((0, bar_height_px + 2), "0", fill="black", font=font)
    end_label = f"{target_cm} cm"
    end_label_bbox = draw.textbbox((0, 0), end_label, font=font)
    end_label_width = end_label_bbox[2] - end_label_bbox[0]
    draw.text((bar_width_px - end_label_width, bar_height_px + 2), end_label, fill="black", font=font)
    
    return bar_img

# ... (place_images_grid, place_images_puzzle, save_output, run_layout_process sono identiche alla versione precedente)
def place_images_grid(image_data, page_size_px, grid_size, margin_px, spacing_px, status_callback=print):
    rows_per_page, suggested_cols = grid_size
    page_width, page_height = page_size_px
    available_width = page_width - (2 * margin_px)
    pages, image_index = [], 0
    while image_index < len(image_data):
        current_page = Image.new('RGB', page_size_px, 'white')
        current_y = margin_px
        page_has_images = False
        rows_on_this_page = 0
        while image_index < len(image_data) and rows_on_this_page < rows_per_page:
            row_images, current_row_width, row_height = [], 0, 0
            temp_index = image_index
            while temp_index < len(image_data) and len(row_images) < suggested_cols:
                img = image_data[temp_index]['img']
                needed_width = current_row_width + img.width + (spacing_px if row_images else 0)
                if needed_width <= available_width:
                    row_images.append(image_data[temp_index])
                    current_row_width = needed_width
                    row_height = max(row_height, img.height)
                    temp_index += 1
                else: break
            if not row_images and image_index < len(image_data):
                row_images.append(image_data[image_index])
                row_height = image_data[image_index]['img'].height
                temp_index = image_index + 1
            if not row_images: break
            if current_y + row_height > page_height - margin_px:
                image_index = temp_index - len(row_images); break
            image_index = temp_index
            total_row_img_width = sum(d['img'].width for d in row_images)
            total_row_width_with_spacing = total_row_img_width + spacing_px * (len(row_images) - 1)
            start_x = margin_px + (available_width - total_row_width_with_spacing) // 2
            current_x = start_x
            for img_data in row_images:
                img = img_data['img']
                paste_y = current_y + (row_height - img.height) // 2
                current_page.paste(img, (current_x, paste_y), img if img.mode == 'RGBA' else None)
                current_x += img.width + spacing_px
                page_has_images = True
            current_y += row_height + spacing_px
            rows_on_this_page += 1
        if page_has_images: pages.append(current_page)
        if not page_has_images and image_index < len(image_data):
            status_callback("ATTENZIONE: Immagini rimanenti potrebbero essere troppo grandi."); break
    return pages

def place_images_puzzle(image_data, page_size_px, margin_px, spacing_px, status_callback=print):
    page_width, page_height = page_size_px
    bin_width, bin_height = page_width - (2 * margin_px), page_height - (2 * margin_px)
    packer = rectpack.newPacker(rotation=False)
    images = [d['img'] for d in image_data]
    for i, img in enumerate(images): packer.add_rect(img.width + spacing_px, img.height + spacing_px, rid=i)
    for _ in range(len(images)): packer.add_bin(bin_width, bin_height)
    packer.pack()
    pages = []
    for i, abin in enumerate(packer):
        if not abin: break
        page = Image.new('RGB', page_size_px, 'white')
        status_callback(f"Creazione pagina puzzle {i+1}...")
        for rect in abin:
            original_image = images[rect.rid]
            paste_x, paste_y = margin_px + rect.x, margin_px + rect.y
            page.paste(original_image, (paste_x, paste_y), original_image if original_image.mode == 'RGBA' else None)
        pages.append(page)
    return pages

def save_output(pages, output_file, output_dpi, status_callback=print):
    if not pages: status_callback("Nessuna pagina generata."); return
    output_path = Path(output_file)
    output_path.parent.mkdir(parents=True, exist_ok=True)
    if pages and not pages[0].mode == 'RGB': pages = [p.convert('RGB') for p in pages]
    
    status_callback(f"Salvataggio output in: {output_path.resolve()} con metadati a {output_dpi} DPI.")
    if output_path.suffix.lower() == '.pdf':
        pages[0].save(output_path, "PDF", resolution=float(output_dpi), save_all=True, append_images=pages[1:])
    else:
        if len(pages) > 1:
            base, ext = output_path.stem, output_path.suffix
            for i, page in enumerate(pages): page.save(output_path.with_name(f"{base}_{i+1}{ext}"), dpi=(output_dpi, output_dpi))
        else:
            pages[0].save(output_path, dpi=(output_dpi, output_dpi))

def run_layout_process(params, status_callback=print):
    """Funzione principale che esegue l'intero processo di layout."""
    try:
        metadata = load_metadata(params['metadata_file'], status_callback)
        image_data = load_images_with_info(params['input_folder'], status_callback)
        if not image_data: status_callback("Nessuna immagine trovata."); return
        sort_by = params['sort_by']
        status_callback(f"Ordinamento immagini tramite: '{sort_by or 'alfabetico'}'...")
        if sort_by == "random": random.shuffle(image_data)
        elif sort_by == "natural_name": image_data.sort(key=lambda d: natural_sort_key(d['name']))
        elif sort_by and sort_by not in ["", "alphabetical", "random", "natural_name"] and metadata:
            image_data.sort(key=lambda d: (str(metadata.get(d['name'], {}).get(sort_by, 'zz_fallback')), natural_sort_key(d['name'])))
        
        image_data = scale_images(image_data, params['scale_factor'], status_callback)
        
        if params['add_caption']:
            image_data = add_captions_to_images(image_data, metadata, params['caption_font_size'], params['caption_padding'], status_callback)
        
        page_dims = get_page_dimensions(params['page_size_px'])
        
        final_pages = []
        status_callback(f"Avvio posizionamento in modalità '{params['mode']}'...")
        if params['mode'] == 'grid':
            grid_size = (params['grid_rows'], params['grid_cols'])
            final_pages = place_images_grid(image_data, page_dims, grid_size, params['margin_px'], params['spacing_px'], status_callback)
        elif params['mode'] == 'puzzle':
            final_pages = place_images_puzzle(image_data, page_dims, params['margin_px'], params['spacing_px'], status_callback)
        
        status_callback(f"Generate {len(final_pages)} pagine.")

        if params['add_scale_bar'] and final_pages:
            # Chiama la nuova versione di create_scale_bar
            scale_bar = create_scale_bar(params['scale_bar_cm'], params['pixels_per_cm'], params['scale_factor'], status_callback)
            for page in final_pages:
                x_pos, y_pos = params['margin_px'], page.height - params['margin_px'] - scale_bar.height
                page.paste(scale_bar, (x_pos, y_pos), scale_bar)

        save_output(final_pages, params['output_file'], params['output_dpi'], status_callback)
        status_callback("--- PROCESSO COMPLETATO CON SUCCESSO ---")

    except Exception as e:
        status_callback(f"--- ERRORE: {e} ---")
        raise e

# =============================================================================
# FUNZIONE DI TEST CON LOGICA DELLA SCALA CORRETTA
# =============================================================================
def test_pixel_workflow():
    print("--- ESECUZIONE TEST DI LAYOUT CON SCALA ACCURATA ---")
    
    # --- 1. Definisci la scala delle tue immagini sorgenti ---
    # !!! QUESTO È IL PARAMETRO PIÙ IMPORTANTE PER LA SCALA GRAFICA !!!
    # Misura in una tua immagine originale quanti pixel occupa 1 cm reale.
    PIXELS_PER_CM = 127  # Esempio: se un oggetto di 1cm è largo 116px nella foto.

    # --- 2. Definisci gli altri parametri ---
    INPUT_FOLDER = "./immagini" 
    OUTPUT_FILE = "risultato_test_scala_corretta.pdf"
    
    PAGE_SIZE_PX = "2480x3508"
    MODE = "grid"
    SORT_BY = "natural_name"
    
    SCALE_FACTOR = 0.5  # Riduciamo tutto al 50%
    MARGIN_PX = 100
    SPACING_PX = 20
    
    GRID_ROWS = 8
    GRID_COLS = 5

    ADD_CAPTION = True
    ADD_SCALE_BAR = True
    SCALE_BAR_CM = 5  # Vogliamo una barra che rappresenti 5 cm reali
    
    # Questo influenza solo i metadati del file finale, non i calcoli della scala.
    OUTPUT_DPI = 300
    
    # --- Dizionario dei parametri (non modificare) ---
    params = {
        'input_folder': INPUT_FOLDER, 'output_file': OUTPUT_FILE,
        'metadata_file': "", 'page_size_px': PAGE_SIZE_PX,
        'mode': MODE, 'sort_by': SORT_BY, 'scale_factor': SCALE_FACTOR,
        'margin_px': MARGIN_PX, 'spacing_px': SPACING_PX,
        'grid_rows': GRID_ROWS, 'grid_cols': GRID_COLS,
        'add_caption': ADD_CAPTION, 'caption_font_size': 10, 'caption_padding': 4,
        'add_scale_bar': ADD_SCALE_BAR, 'scale_bar_cm': SCALE_BAR_CM,
        'pixels_per_cm': PIXELS_PER_CM, # Il nuovo parametro fondamentale
        'output_dpi': OUTPUT_DPI
    }

    run_layout_process(params)

if __name__ == "__main__":
    test_pixel_workflow()

--- ESECUZIONE TEST DI LAYOUT CON SCALA ACCURATA ---
Caricamento immagini da: ./immagini...
Caricati 24 immagini.
Ordinamento immagini tramite: 'natural_name'...
Applicazione scala: 0.5x
Aggiunta didascalie alle immagini...
Avvio posizionamento in modalità 'grid'...
Generate 7 pagine.
Creazione scala grafica per rappresentare 5 cm...
Salvataggio output in: C:\Users\larth\Documents\PyPotteryLayout\risultato_test_scala_corretta.pdf con metadati a 300 DPI.
--- PROCESSO COMPLETATO CON SUCCESSO ---


In [10]:
import os
import argparse
from pathlib import Path
from PIL import Image, ImageDraw, ImageFont
import rectpack

# =============================================================================
# FUNZIONI LOGICHE (NON MODIFICARE)
# =============================================================================

PAGE_SIZES_MM = {'A4': (210, 297), 'A3': (297, 420)}

def mm_to_pixels(mm, dpi):
    return int((mm / 25.4) * dpi)

def get_page_dimensions_px(size_name, dpi, custom_size_str):
    if size_name.lower() == 'custom':
        if not custom_size_str: raise ValueError("Specificare --custom-size LARGHEZZAxALTEZZA.")
        try:
            width, height = map(int, custom_size_str.split('x'))
            return (width, height)
        except ValueError: raise ValueError("Formato --custom-size non valido.")
    size_mm = PAGE_SIZES_MM.get(size_name.upper())
    if not size_mm: raise ValueError(f"Formato pagina non supportato: {size_name}.")
    return (mm_to_pixels(size_mm[0], dpi), mm_to_pixels(size_mm[1], dpi))

def load_images_with_info(folder_path):
    """Carica immagini e conserva i loro nomi file."""
    image_data = []
    supported_formats = ('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff')
    print(f"Caricamento immagini da: {folder_path}...")
    if not os.path.isdir(folder_path):
        raise FileNotFoundError(f"La cartella di input '{folder_path}' non esiste.")
    for filename in sorted(os.listdir(folder_path)):
        if filename.lower().endswith(supported_formats):
            try:
                filepath = os.path.join(folder_path, filename)
                img = Image.open(filepath)
                image_data.append({'img': img.copy(), 'name': filename})
                img.close()
            except IOError: print(f"Attenzione: Impossibile caricare il file {filename}.")
    print(f"Caricati {len(image_data)} immagini.")
    return image_data

def scale_images(image_data, scale_factor):
    if scale_factor == 1.0: return image_data
    print(f"Applicazione di un fattore di scala di {scale_factor}...")
    for data in image_data:
        new_width = int(data['img'].width * scale_factor)
        new_height = int(data['img'].height * scale_factor)
        data['img'] = data['img'].resize((new_width, new_height), Image.Resampling.LANCZOS)
    return image_data

def add_captions_to_images(image_data, font_size, caption_padding):
    """Aggiunge una didascalia con il nome del file sotto ogni immagine."""
    print("Aggiunta didascalie alle immagini...")
    try:
        # Cerca un font comune, altrimenti usa quello di default
        font = ImageFont.truetype("arial.ttf", font_size)
    except IOError:
        print("Attenzione: Font 'Arial' non trovato. Uso il font di default.")
        font = ImageFont.load_default()

    for data in image_data:
        img = data['img']
        caption_text = data['name']
        
        # Crea un contesto di disegno temporaneo per misurare il testo
        draw = ImageDraw.Draw(Image.new('RGB', (1, 1)))
        text_bbox = draw.textbbox((0, 0), caption_text, font=font)
        text_height = text_bbox[3] - text_bbox[1]
        
        # Crea una nuova immagine più alta per contenere l'originale e la didascalia
        new_height = img.height + text_height + caption_padding * 2
        captioned_img = Image.new('RGB', (img.width, new_height), 'white')
        
        # Incolla l'immagine originale
        captioned_img.paste(img, (0, 0))
        
        # Disegna il testo della didascalia, centrato orizzontalmente
        draw = ImageDraw.Draw(captioned_img)
        text_width = text_bbox[2] - text_bbox[0]
        text_x = (img.width - text_width) / 2
        text_y = img.height + caption_padding
        draw.text((text_x, text_y), caption_text, font=font, fill="black")
        
        # Sostituisci l'immagine originale con quella con didascalia
        data['img'] = captioned_img
        
    return image_data

def place_images_grid(image_data, page_size_px, grid_size, margin_px, spacing_px):
    rows, cols = grid_size
    page_width, page_height = page_size_px
    
    # Calcola la dimensione di ogni cella tenendo conto dello spazio tra di esse
    drawable_width = page_width - (2 * margin_px) - ((cols - 1) * spacing_px)
    drawable_height = page_height - (2 * margin_px) - ((rows - 1) * spacing_px)
    cell_width = drawable_width / cols
    cell_height = drawable_height / rows
    
    if cell_width <= 0 or cell_height <= 0:
        raise ValueError("Margini e spaziatura sono troppo grandi per la pagina. Riduci i valori o il numero di colonne/righe.")

    pages, image_iterator, is_done = [], iter(image_data), False
    while not is_done:
        current_page, page_is_filled = Image.new('RGB', page_size_px, 'white'), False
        for r in range(rows):
            for c in range(cols):
                try:
                    data = next(image_iterator)
                    img = data['img'].copy()
                    
                    # --- FIX SOVRAPPOSIZIONE ---
                    # Se l'immagine è più grande della cella, la ridimensiona mantenendo le proporzioni
                    img.thumbnail((cell_width, cell_height), Image.Resampling.LANCZOS)
                    
                    # Calcola la posizione della cella
                    cell_x = margin_px + c * (cell_width + spacing_px)
                    cell_y = margin_px + r * (cell_height + spacing_px)
                    
                    # Centra l'immagine (ormai ridimensionata) all'interno della sua cella
                    paste_x = int(cell_x + (cell_width - img.width) / 2)
                    paste_y = int(cell_y + (cell_height - img.height) / 2)
                    
                    current_page.paste(img, (paste_x, paste_y), img if img.mode == 'RGBA' else None)
                    page_is_filled = True
                except StopIteration: is_done = True; break
            if is_done: break
        if page_is_filled: pages.append(current_page)
    return pages

def place_images_puzzle(image_data, page_size_px, margin_px, spacing_px):
    page_width, page_height = page_size_px
    bin_width, bin_height = page_width - (2 * margin_px), page_height - (2 * margin_px)
    packer = rectpack.newPacker(rotation=False)
    
    images = [d['img'] for d in image_data]
    
    # Aggiunge la spaziatura alla dimensione di ogni rettangolo per il packer
    for i, img in enumerate(images):
        packer.add_rect(img.width + spacing_px, img.height + spacing_px, rid=i)
        
    for _ in range(len(images)): packer.add_bin(bin_width, bin_height)
    packer.pack()
    
    pages = []
    for i, abin in enumerate(packer):
        if not abin: break
        page = Image.new('RGB', page_size_px, 'white')
        print(f"Creazione pagina {i+1} in modalità puzzle...")
        for rect in abin:
            original_image = images[rect.rid]
            # Le coordinate del packer includono già lo spazio, quindi incolliamo l'immagine originale
            paste_x, paste_y = margin_px + rect.x, margin_px + rect.y
            page.paste(original_image, (paste_x, paste_y), original_image if original_image.mode == 'RGBA' else None)
        pages.append(page)
    return pages

def save_output(pages, output_file):
    if not pages: print("Nessuna pagina generata."); return
    # ... (Stessa funzione di salvataggio di prima)
    output_path = Path(output_file)
    output_path.parent.mkdir(parents=True, exist_ok=True)
    if output_path.suffix.lower() == '.pdf':
        pages[0].save(output_path, "PDF", resolution=100.0, save_all=True, append_images=pages[1:])
        print(f"File PDF salvato come: {output_path}")
    else:
        if len(pages) > 1:
            base, ext = output_path.stem, output_path.suffix
            for i, page in enumerate(pages):
                new_filename = output_path.with_name(f"{base}_{i+1}{ext}")
                page.save(new_filename)
                print(f"Pagina {i+1} salvata come: {new_filename}")
        else:
            pages[0].save(output_path)
            print(f"File immagine salvato come: {output_path}")

# =============================================================================
# FUNZIONE DI TEST (MODIFICA QUESTI PARAMETRI)
# =============================================================================
def esegui_test():
    print("--- ESECUZIONE TEST DI LAYOUT ---")
    
    # --- Parametri Generali ---
    INPUT_FOLDER = r"C:\Users\larth\Documents\PyPotteryLens\outputs\ANGL"
    PAGE_SIZE = "A4"
    SCALE_FACTOR = 0.5         # Riduciamo un po' le immagini in partenza
    DPI = 300
    MARGIN_PX = 250
    
    # --- NUOVI PARAMETRI ---
    SPACING_PX = 50            # Spazio in pixel tra le immagini
    ADD_CAPTION = True         # True per aggiungere il nome del file, False per ometterlo
    CAPTION_FONT_SIZE = 15     # Dimensione del font per la didascalia
    CAPTION_PADDING = 5        # Spazio sopra e sotto il testo della didascalia

    try:
        # 1. Caricamento
        image_data = load_images_with_info(INPUT_FOLDER)
        if not image_data: return

        # 2. Scalatura
        image_data = scale_images(image_data, SCALE_FACTOR)

        # 3. Aggiunta Didascalie (opzionale)
        if ADD_CAPTION:
            image_data = add_captions_to_images(image_data, CAPTION_FONT_SIZE, CAPTION_PADDING)

        page_dims = get_page_dimensions_px(PAGE_SIZE, DPI, "1200x800")
        
        # --- Test 1: Modalità GRID (Corretta) ---
        print("\n--- TEST 1: Modalità GRID ---")
        GRID_ROWS = 3
        GRID_COLS = 2
        OUTPUT_FILE_GRID = "test_grid_con_spazio_e_didascalie.pdf"
        
        grid_pages = place_images_grid(image_data, page_dims, (GRID_ROWS, GRID_COLS), MARGIN_PX, SPACING_PX)
        save_output(grid_pages, OUTPUT_FILE_GRID)

        # --- Test 2: Modalità PUZZLE ---
        print("\n--- TEST 2: Modalità PUZZLE ---")
        OUTPUT_FILE_PUZZLE = "test_puzzle_con_spazio_e_didascalie.pdf"
        
        puzzle_pages = place_images_puzzle(image_data, page_dims, MARGIN_PX, SPACING_PX)
        save_output(puzzle_pages, OUTPUT_FILE_PUZZLE)

    except (ValueError, FileNotFoundError) as e:
        print(f"\nERRORE DURANTE IL TEST: {e}")
    
    print("\n--- TEST COMPLETATO ---")

# =============================================================================
# PUNTO DI INGRESSO DELLO SCRIPT
# =============================================================================
if __name__ == "__main__":
    esegui_test()

    # Per riattivare la riga di comando, commenta esegui_test() e implementa
    # la funzione main() con argparse che includa i nuovi argomenti:
    # --spacing (int), --add-caption (flag), --caption-font-size (int), ecc.

--- ESECUZIONE TEST DI LAYOUT ---
Caricamento immagini da: C:\Users\larth\Documents\PyPotteryLens\outputs\ANGL...
Caricati 120 immagini.
Applicazione di un fattore di scala di 0.5...
Aggiunta didascalie alle immagini...

--- TEST 1: Modalità GRID ---
File PDF salvato come: test_grid_con_spazio_e_didascalie.pdf

--- TEST 2: Modalità PUZZLE ---
Creazione pagina 1 in modalità puzzle...
Creazione pagina 2 in modalità puzzle...
Creazione pagina 3 in modalità puzzle...
File PDF salvato come: test_puzzle_con_spazio_e_didascalie.pdf

--- TEST COMPLETATO ---


In [3]:
# gui_app.py

import tkinter as tk
from tkinter import ttk, filedialog, messagebox
from tkinter.scrolledtext import ScrolledText
import ttkbootstrap as ttk
from ttkbootstrap.constants import *
import threading
import os

# Assicurati che backend_logic.py sia nella stessa cartella
import backend_logic

class LayoutApp(ttk.Window):
    def __init__(self):
        super().__init__(themename="litera")
        self.title("Image Layout Tool")
        self.geometry("800x900")

        # Variabili di stato per i widget
        self.vars = {
            'input_folder': tk.StringVar(),
            'output_file': tk.StringVar(),
            'metadata_file': tk.StringVar(),
            'mode': tk.StringVar(value="grid"),
            'page_size': tk.StringVar(value="A4"),
            'custom_size': tk.StringVar(),  # Nuovo: per dimensioni personalizzate
            'sort_by': tk.StringVar(value="alphabetical"),
            'scale_factor': tk.DoubleVar(value=1.0),
            'margin_px': tk.IntVar(value=50),
            'spacing_px': tk.IntVar(value=10),
            'grid_rows': tk.IntVar(value=4),
            'grid_cols': tk.IntVar(value=3),
            'add_caption': tk.BooleanVar(value=True),
            'caption_font_size': tk.IntVar(value=12),
            'caption_padding': tk.IntVar(value=5),
            'add_scale_bar': tk.BooleanVar(value=True),
            'scale_bar_length_px': tk.IntVar(value=200),  # Aggiornato: lunghezza in pixel
            'scale_bar_real_size_text': tk.StringVar(value="5 cm"),  # Nuovo: testo della misura reale
        }

        self._create_widgets()
        self._update_ui_for_mode()
        self._update_sort_options()
        self._update_page_size_options()

    def _create_widgets(self):
        main_frame = ttk.Frame(self, padding="10")
        main_frame.pack(fill=BOTH, expand=YES)

        # --- Sezione File & Cartelle ---
        path_frame = ttk.Labelframe(main_frame, text="1. File e Cartelle", padding="10")
        path_frame.pack(fill=X, expand=NO, pady=5)
        self._create_path_widgets(path_frame)

        # --- Sezione Layout ---
        layout_frame = ttk.Labelframe(main_frame, text="2. Impostazioni di Layout", padding="10")
        layout_frame.pack(fill=X, expand=NO, pady=5)
        self._create_layout_widgets(layout_frame)

        # --- Sezione Dettagli ---
        details_frame = ttk.Labelframe(main_frame, text="3. Dettagli e Aggiunte", padding="10")
        details_frame.pack(fill=X, expand=NO, pady=5)
        self._create_details_widgets(details_frame)
        
        # --- Pulsante di Avvio ---
        self.run_button = ttk.Button(main_frame, text="Avvia Processo di Layout", command=self._start_process, bootstyle=SUCCESS)
        self.run_button.pack(pady=20, fill=X, ipady=10)

        # --- Log di Stato ---
        log_frame = ttk.Labelframe(main_frame, text="Log di Processo", padding="10")
        log_frame.pack(fill=BOTH, expand=YES, pady=5)
        self.log_text = ScrolledText(log_frame, height=10, wrap=tk.WORD, state='disabled')
        self.log_text.pack(fill=BOTH, expand=YES)

    def _create_path_widgets(self, parent):
        # Input Folder
        ttk.Label(parent, text="Cartella Immagini:").grid(row=0, column=0, sticky=W, padx=5, pady=2)
        ttk.Entry(parent, textvariable=self.vars['input_folder'], state='readonly').grid(row=0, column=1, sticky=EW, padx=5)
        ttk.Button(parent, text="Sfoglia...", command=self._browse_input_folder).grid(row=0, column=2, padx=5)
        # Output File
        ttk.Label(parent, text="File di Output:").grid(row=1, column=0, sticky=W, padx=5, pady=2)
        ttk.Entry(parent, textvariable=self.vars['output_file'], state='readonly').grid(row=1, column=1, sticky=EW, padx=5)
        ttk.Button(parent, text="Salva come...", command=self._browse_output_file).grid(row=1, column=2, padx=5)
        # Metadata File
        ttk.Label(parent, text="File Metadati (opz.):").grid(row=2, column=0, sticky=W, padx=5, pady=2)
        ttk.Entry(parent, textvariable=self.vars['metadata_file'], state='readonly').grid(row=2, column=1, sticky=EW, padx=5)
        ttk.Button(parent, text="Sfoglia...", command=self._browse_metadata_file).grid(row=2, column=2, padx=5)
        parent.columnconfigure(1, weight=1)

    def _create_layout_widgets(self, parent):
        left_frame = ttk.Frame(parent)
        left_frame.grid(row=0, column=0, sticky=NSEW, padx=5)
        right_frame = ttk.Frame(parent)
        right_frame.grid(row=0, column=1, sticky=NSEW, padx=5, ipadx=20)
        parent.columnconfigure(0, weight=1)
        parent.columnconfigure(1, weight=1)

        # Colonna Sinistra
        ttk.Label(left_frame, text="Modalità:").pack(anchor=W, pady=(0,2))
        ttk.Radiobutton(left_frame, text="Griglia (Grid)", variable=self.vars['mode'], value="grid", command=self._update_ui_for_mode).pack(anchor=W)
        ttk.Radiobutton(left_frame, text="Puzzle (Ottimizzato)", variable=self.vars['mode'], value="puzzle", command=self._update_ui_for_mode).pack(anchor=W)

        ttk.Label(left_frame, text="Formato Pagina:").pack(anchor=W, pady=(10,2))
        self.page_size_combo = ttk.Combobox(left_frame, textvariable=self.vars['page_size'], 
                                           values=["A4", "A3", "HD", "4K", "LETTER", "custom"], 
                                           state='readonly')
        self.page_size_combo.pack(fill=X)
        self.page_size_combo.bind('<<ComboboxSelected>>', self._update_page_size_options)
        
        # Frame per dimensioni personalizzate
        self.custom_size_frame = ttk.Frame(left_frame)
        self.custom_size_frame.pack(fill=X, pady=(5,0))
        ttk.Label(self.custom_size_frame, text="Dimensioni (WxH):").pack(anchor=W)
        custom_entry = ttk.Entry(self.custom_size_frame, textvariable=self.vars['custom_size'])
        custom_entry.pack(fill=X)
        
        ttk.Label(left_frame, text="Ordinamento Immagini:").pack(anchor=W, pady=(10,2))
        self.sort_by_combo = ttk.Combobox(left_frame, textvariable=self.vars['sort_by'], state='readonly')
        self.sort_by_combo.pack(fill=X)

        # Colonna Destra (Grid-specific)
        self.grid_frame = ttk.Frame(right_frame)
        self.grid_frame.pack(fill=X)
        ttk.Label(self.grid_frame, text="Righe Griglia:").grid(row=0, column=0, sticky=W)
        ttk.Spinbox(self.grid_frame, from_=1, to=100, textvariable=self.vars['grid_rows']).grid(row=0, column=1, sticky=EW, padx=5)
        ttk.Label(self.grid_frame, text="Colonne Griglia:").grid(row=1, column=0, sticky=W)
        ttk.Spinbox(self.grid_frame, from_=1, to=100, textvariable=self.vars['grid_cols']).grid(row=1, column=1, sticky=EW, padx=5)
        self.grid_frame.columnconfigure(1, weight=1)
        
    def _create_details_widgets(self, parent):
        # Creo una griglia 2x2 per organizzare meglio i controlli
        f1 = ttk.Frame(parent)
        f1.grid(row=0, column=0, sticky=NSEW, padx=5)
        f2 = ttk.Frame(parent)
        f2.grid(row=0, column=1, sticky=NSEW, padx=5)
        f3 = ttk.Frame(parent)
        f3.grid(row=1, column=0, sticky=NSEW, padx=5, pady=(10,0))
        f4 = ttk.Frame(parent)
        f4.grid(row=1, column=1, sticky=NSEW, padx=5, pady=(10,0))
        
        parent.columnconfigure(0, weight=1)
        parent.columnconfigure(1, weight=1)
        parent.rowconfigure(0, weight=1)
        parent.rowconfigure(1, weight=1)
        
        # Frame 1: Scala Immagini
        ttk.Label(f1, text="Scala Immagini:").pack(anchor=W)
        scale_frame = ttk.Frame(f1)
        scale_frame.pack(fill=X, pady=2)
        ttk.Scale(scale_frame, from_=0.1, to=3.0, variable=self.vars['scale_factor'], orient=HORIZONTAL).pack(side=LEFT, fill=X, expand=YES)
        self.scale_label = ttk.Label(scale_frame, text="1.0", width=5)
        self.scale_label.pack(side=RIGHT, padx=(5,0))
        
        # Binding per aggiornare il label della scala
        self.vars['scale_factor'].trace('w', self._update_scale_label)
        
        # Frame 2: Margini e Spaziatura
        ttk.Label(f2, text="Margine Pagina (px):").pack(anchor=W)
        ttk.Spinbox(f2, from_=0, to=500, textvariable=self.vars['margin_px']).pack(fill=X, pady=2)
        
        ttk.Label(f2, text="Spazio tra Immagini (px):").pack(anchor=W, pady=(10,0))
        ttk.Spinbox(f2, from_=0, to=200, textvariable=self.vars['spacing_px']).pack(fill=X, pady=2)
        
        # Frame 3: Didascalie
        ttk.Checkbutton(f3, text="Aggiungi Didascalie", variable=self.vars['add_caption'], command=self._update_caption_options).pack(anchor=W, pady=(0,2))
        
        # Frame per le opzioni delle didascalie
        self.caption_options_frame = ttk.Frame(f3)
        self.caption_options_frame.pack(fill=X, padx=20)
        ttk.Label(self.caption_options_frame, text="Dimensione Font:").grid(row=0, column=0, sticky=W, pady=2)
        ttk.Spinbox(self.caption_options_frame, from_=8, to=24, textvariable=self.vars['caption_font_size'], width=5).grid(row=0, column=1, sticky=W, padx=5)
        ttk.Label(self.caption_options_frame, text="Padding:").grid(row=1, column=0, sticky=W, pady=2)
        ttk.Spinbox(self.caption_options_frame, from_=0, to=50, textvariable=self.vars['caption_padding'], width=5).grid(row=1, column=1, sticky=W, padx=5)
        self.caption_options_frame.columnconfigure(1, weight=1)

        # Frame 4: Scala Grafica
        ttk.Checkbutton(f4, text="Aggiungi Scala Grafica", variable=self.vars['add_scale_bar'], command=self._update_scale_bar_options).pack(anchor=W, pady=2)
        
        # Frame per le opzioni della scala grafica
        self.scale_bar_options_frame = ttk.Frame(f4)
        self.scale_bar_options_frame.pack(fill=X, padx=20, pady=(5,0))
        ttk.Label(self.scale_bar_options_frame, text="Lunghezza (px):").grid(row=0, column=0, sticky=W, pady=2)
        ttk.Spinbox(self.scale_bar_options_frame, from_=50, to=1000, textvariable=self.vars['scale_bar_length_px'], width=8).grid(row=0, column=1, sticky=W, padx=5)
        ttk.Label(self.scale_bar_options_frame, text="Misura reale:").grid(row=1, column=0, sticky=W, pady=2)
        ttk.Entry(self.scale_bar_options_frame, textvariable=self.vars['scale_bar_real_size_text'], width=8).grid(row=1, column=1, sticky=W, padx=5)
        self.scale_bar_options_frame.columnconfigure(1, weight=1)
        
        # Info box
        info_frame = ttk.Frame(f4)
        info_frame.pack(fill=X, pady=(10,0))
        info_text = ttk.Label(info_frame, text="ℹ️ Es: '5 cm', '10 mm', '2 inches'", 
                             font=('TkDefaultFont', 8), foreground='blue')
        info_text.pack(anchor=W)
        
        # Aggiorna lo stato iniziale delle opzioni
        self._update_caption_options()
        self._update_scale_bar_options()

    def _update_scale_label(self, *args):
        """Aggiorna il label che mostra il valore della scala"""
        value = self.vars['scale_factor'].get()
        self.scale_label.config(text=f"{value:.2f}")

    def _update_page_size_options(self, event=None):
        """Mostra/nasconde il campo per dimensioni personalizzate"""
        if self.vars['page_size'].get().lower() == 'custom':
            self.custom_size_frame.pack(fill=X, pady=(5,0))
        else:
            self.custom_size_frame.pack_forget()

    def _update_caption_options(self):
        """Abilita/disabilita le opzioni delle didascalie"""
        if self.vars['add_caption'].get():
            for widget in self.caption_options_frame.winfo_children():
                widget.configure(state='normal')
        else:
            for widget in self.caption_options_frame.winfo_children():
                if hasattr(widget, 'configure'):
                    widget.configure(state='disabled')

    def _update_scale_bar_options(self):
        """Abilita/disabilita le opzioni della scala grafica"""
        if self.vars['add_scale_bar'].get():
            for widget in self.scale_bar_options_frame.winfo_children():
                widget.configure(state='normal')
        else:
            for widget in self.scale_bar_options_frame.winfo_children():
                if hasattr(widget, 'configure'):
                    widget.configure(state='disabled')

    def _browse_input_folder(self):
        folder = filedialog.askdirectory(title="Seleziona la Cartella delle Immagini")
        if folder: 
            self.vars['input_folder'].set(folder)

    def _browse_output_file(self):
        file = filedialog.asksaveasfilename(title="Salva il File di Output", defaultextension=".pdf", 
                                          filetypes=[("PDF Document", "*.pdf"), ("PNG Image", "*.png"), ("All Files", "*.*")])
        if file: 
            self.vars['output_file'].set(file)

    def _browse_metadata_file(self):
        file = filedialog.askopenfilename(title="Seleziona il File Metadati", filetypes=[("Excel Files", "*.xlsx")])
        if file:
            self.vars['metadata_file'].set(file)
            self._update_sort_options()

    def _update_sort_options(self):
        """Aggiorna le opzioni di ordinamento basandosi sui metadati disponibili"""
        metadata_file = self.vars['metadata_file'].get()
        options = ["alphabetical", "random", "natural_name"]
        if metadata_file and os.path.exists(metadata_file):
            headers = backend_logic.get_metadata_headers(metadata_file)
            if headers and len(headers) > 1:
                # Escludi la prima colonna (filename) e aggiungi le altre come opzioni di sort
                options.extend(headers[1:])
        
        self.sort_by_combo['values'] = options
        if self.vars['sort_by'].get() not in options:
            self.vars['sort_by'].set("alphabetical")

    def _update_ui_for_mode(self):
        """Mostra/nasconde i controlli specifici per la modalità griglia"""
        if self.vars['mode'].get() == "grid":
            self.grid_frame.pack(fill=X)
        else:
            self.grid_frame.pack_forget()

    def _update_log(self, message):
        """Aggiunge un messaggio al log di processo"""
        self.log_text.config(state='normal')
        self.log_text.insert(tk.END, message + "\n")
        self.log_text.see(tk.END)
        self.log_text.config(state='disabled')
        self.update_idletasks()

    def _validate_inputs(self, params):
        """Valida i parametri di input prima di avviare il processo"""
        if not params['input_folder'] or not os.path.exists(params['input_folder']):
            return "Seleziona una cartella di input valida."
        
        if not params['output_file']:
            return "Specifica un file di output."
            
        if params['page_size'].lower() == 'custom':
            if not params['custom_size']:
                return "Specifica le dimensioni personalizzate (es. 1920x1080)."
            try:
                w, h = params['custom_size'].split('x')
                int(w), int(h)
            except:
                return "Formato dimensioni personalizzate non valido. Usa: WIDTHxHEIGHT"
        
        if params['scale_factor'] <= 0:
            return "Il fattore di scala deve essere maggiore di 0."
            
        if params['add_scale_bar'] and not params['scale_bar_real_size_text']:
            return "Specifica la misura reale per la scala grafica."
            
        return None

    def _start_process(self):
        """Avvia il processo di layout in un thread separato"""
        # Ottieni i parametri dalle variabili
        params = {key: var.get() for key, var in self.vars.items()}
        
        # Validazione input
        error_msg = self._validate_inputs(params)
        if error_msg:
            messagebox.showerror("Errore", error_msg)
            return
        
        # Disabilita il pulsante e pulisci il log
        self.run_button.config(state='disabled', text="Elaborazione in corso...")
        self.log_text.config(state='normal')
        self.log_text.delete('1.0', tk.END)
        self.log_text.config(state='disabled')

        # Avvia il processo in un thread separato
        thread = threading.Thread(target=self._run_backend_in_thread, args=(params,))
        thread.daemon = True
        thread.start()

    def _run_backend_in_thread(self, params):
        """Esegue il backend in un thread separato"""
        try:
            # Converti i parametri nel formato atteso dal backend
            backend_params = self._convert_params_for_backend(params)
            
            # Esegui il processo
            backend_logic.run_layout_process(backend_params, self._update_log)
            
            # Mostra messaggio di successo
            self.after(0, lambda: messagebox.showinfo("Successo", 
                f"Processo completato con successo!\nFile salvato in:\n{params['output_file']}"))
                
        except Exception as e:
            self._update_log(f"ERRORE CRITICO: {e}")
            self.after(0, lambda: messagebox.showerror("Errore", 
                f"Si è verificato un errore durante il processo:\n\n{str(e)}"))
        finally:
            # Riabilita il pulsante
            self.after(0, lambda: self.run_button.config(state='normal', text="Avvia Processo di Layout"))

    def _convert_params_for_backend(self, gui_params):
        """Converte i parametri della GUI nel formato atteso dal backend"""
        backend_params = gui_params.copy()
        
        # Converti sort_by se è vuoto o "alphabetical"
        if backend_params['sort_by'] in ['', 'alphabetical']:
            backend_params['sort_by'] = None
            
        return backend_params

if __name__ == "__main__":
    app = LayoutApp()
    app.mainloop()

Caricamento immagini da: C:/Users/larth/Desktop/RAPID...
Caricati 24 immagini.
Applicazione fattore di scala: 0.5009646302250804
Aggiunta didascalie alle immagini...


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\larth\anaconda3\envs\base_new\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "c:\Users\larth\anaconda3\envs\base_new\Lib\tkinter\__init__.py", line 861, in callit
    func(*args)
  File "C:\Users\larth\AppData\Local\Temp\ipykernel_34604\2944026021.py", line 341, in <lambda>
    f"Si è verificato un errore durante il processo:\n\n{str(e)}"))
                                                             ^
NameError: cannot access free variable 'e' where it is not associated with a value in enclosing scope
